# Homework 3

## Inference on Graphical Models

In the previous homework, you constructed and examined the properties of a DAG and a MRF to model the variables for a murder mystery problem. Now, you are ready to perform inference on the directed graphical models. 

The goal of this analysis is to find the posterior distribution of key variables which will help Inspector Markov determine the most likely, weapon, motive and perpetrator. This process will require **inference** on the models. 


*******
In this exercise you will construct a Directed Bayesian Graphical Model or belief network for the available evidence and perform inference on some of the variables. 

Inspector Markov has continued her investigation. Additionally Dr. Turing has had time to perform laboratory analysis. Turing reports to the Inspector that there is a chance that a toxic substance may have been used to incapacitate the victim before the stabbing. So, there are now two possible weapons:
- knife alone, 
- knife with a poison. 

Given this evidence, Inspector Markov must update her beliefs. Further, she needs to perform inference to determine if there are likely combinations of suspects and weapons. 



Recall that the joint probability distribution is:

$$p(B,C,W,MO,M)$$   
where the letters indicate the following variables;   
$B = $ unconditional probability that the butler committed the crime,   
$C = $ unconditional probability that the cook committed the crime,   
$W = $ the probability of the weapon, K = knife, P = poison, conditional on B and C.   
$MO = $ the probability of a motive, conditional on C and B.    
$M = $ the probability that the third party, the cook, C, or the butler, B, committed the crime, conditional on B, C, W, and MO.    

Given the independencies, this distribution can be factorized in the following manner:

$$p(B,C,W,MO,M) = p(B)\ p(C)\ p(W\ |\ B, C)\ p(MO\ |\ B,C)\ p(M\ |\ B, C, W, MO)$$


As a first step in creating the belief network, import the packages you will need for this analysis.

In [2]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

Next, define your directed graphical model in the cell below. You should use the model you defined in Homework 1 and used in Homework 2. Be sure to apply the `check_model` method to you model.

> **Tip:** If you have saved your pickled model you are read it by uncommenting the code below. You will need to change the name `my_model.pickle` to whatever name you have save your model under. 

In [7]:
import pickle
#with open('my_model.pickle', 'rb') as pkl:
#    murder_model = pickle.load(pkl)

#print(murder_model.check_model())
murder_model = BayesianModel([('B','W'),('C','W'),('B','MO'),('C','MO'),('B','M'),('C','M'),('W','M'),('MO','M')])
CPD_B = TabularCPD(variable='B',variable_card=2,values=[[0.4,0.6]])
CPD_C = TabularCPD(variable='C',variable_card=2,values=[[0.7,0.3]])
CPD_W = TabularCPD(variable='W', variable_card=2, values=[[0.1,0.5,0.4,0.7],[0.9,0.5,0.6,0.3]], 
                   evidence=['B','C'], evidence_card=[2,2])
CPD_MO = TabularCPD(variable='MO', variable_card=2, values=[[1,0.7,0.1,0.3],[0,0.3,0.9,0.7]],
                   evidence=['B','C'], evidence_card=[2,2])
CPD_M = TabularCPD(variable='M', variable_card=3, values=[[1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
                                                          [0,0,0,0,1,1,1,1,0,0,0,0,.5,.5,.5,.5],
                                                          [0,0,0,0,0,0,0,0,1,1,1,1,.5,.5,.5,.5]],
                   evidence=['B','C','W','MO'], evidence_card=[2,2,2,2])
murder_model.add_cpds(CPD_B, CPD_C, CPD_W, CPD_MO, CPD_M)
murder_model.check_model()
pickle.dump(murder_model, open("murder_model.pkl", "wb"))

Now that you have the model define, you are ready to apply **variable elimination** to perform inference. In the cell below create and execute code to do the following:    
1. Create an inference object using the `VariableElimination` function.   
2. Perform a query on the variable W. 
3. print the query for W. 

You can find [detailed documentation on the VariableELimination methods in the pgmpy documentation](http://pgmpy.org/inference.html). 

In [3]:
from pgmpy.inference import VariableElimination
murder_infer = VariableElimination(m_model)
w_qur = murder_infer.query(variables=['W'])
print(w_qur['W'])

+-----+----------+
| W   |   phi(W) |
+=====+==========+
| W_0 |   0.3820 |
+-----+----------+
| W_1 |   0.6180 |
+-----+----------+


C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


Now perform a query on the motive, MO, variable to find the marginal distribution.   

In [4]:
mo_qur = murder_infer.query(variables=['MO'])
print(mo_qur['MO'])

+------+-----------+
| MO   |   phi(MO) |
+======+===========+
| MO_0 |    0.4600 |
+------+-----------+
| MO_1 |    0.5400 |
+------+-----------+


C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


These marginal distributions quantify the initial beliefs for these variables. Given these marginal distributions, which weapon and motive appears to be the most likely and why? 

ANS: The likely weapon, according to these distributions, is W_1(poison) and that there likely is a motive MO_1.

What Inspector Markov really needs to know is the probability that each of the suspects committed the murder. In the cell below, create the code to make a query and print the results on the butler, B, and cook, C. 

In [5]:
bc_qur = murder_infer.query(variables=['B', 'C'])
print(bc_qur['B'])
print(bc_qur['C'])

+-----+----------+
| B   |   phi(B) |
+=====+==========+
| B_0 |   0.4000 |
+-----+----------+
| B_1 |   0.6000 |
+-----+----------+
+-----+----------+
| C   |   phi(C) |
+=====+==========+
| C_0 |   0.7000 |
+-----+----------+
| C_1 |   0.3000 |
+-----+----------+


It is possible that Inspector Markov could discover evidence which would cause the beliefs to be updated. 

For example, Sgt Bernoulli telephones the Inspector to say that he has discovered that the butler is indeed due the inheritance (MO = 1). Using this evidence compute the query and print the marginal distribution for the butler, B, and cook, C, are murders. These marginal distributions are the updated belief on who the perpetrator might be.   

In [6]:
bc_qur = murder_infer.query(variables=['B', 'C'], evidence={'MO':1})
print(bc_qur['B'])
print(bc_qur['C'])

+-----+----------+
| B   |   phi(B) |
+=====+==========+
| B_0 |   0.0667 |
+-----+----------+
| B_1 |   0.9333 |
+-----+----------+
+-----+----------+
| C   |   phi(C) |
+=====+==========+
| C_0 |   0.7000 |
+-----+----------+
| C_1 |   0.3000 |
+-----+----------+


Notice how the marginal belief has changed for one suspect but not the other. How can you best explain this change, given the new evidence?

ANS: The change in evidence for motive only effects the butler because the butler is due an inheritance and the cook is not.

Next, Dr. Turing, the medical examiner, call to say that she has discovered that poison was definitely used for the murder, or W = 0. Using this evidence compute the updated belief with a query and print the marginal distributions for the butler, B, and cook, C, are murders. 

In [8]:
bc_qur = murder_infer.query(variables=['B', 'C'], evidence={'MO':1, 'W':0})
print(bc_qur['B'])
print(bc_qur['C'])

+-----+----------+
| B   |   phi(B) |
+=====+==========+
| B_0 |   0.0699 |
+-----+----------+
| B_1 |   0.9301 |
+-----+----------+
+-----+----------+
| C   |   phi(C) |
+=====+==========+
| C_0 |   0.5874 |
+-----+----------+
| C_1 |   0.4126 |
+-----+----------+


Compare the marginal distributions of the cook and the butler being the murderer given the changes in belief from the new evidence of the weapon. How does this change the belief that the cook or the butler are murderers. 

ANS: Poison is the weapon that the cook is more likely to use, that's why the marginal distribution that the cook committed the murder goes up. The fact that there is evidence for motive keeps the marginal distribution the cook committed the murder fairly high and it is dragged down only a little bit by the murder weapon being poison.

You have performed the foregoing analysis using variable elimination. Now, you will perform analysis using the **junction tree algorithm**. 

In the cell below do the following:
1. Create a belief propagation object using the `BeliefPropagation` function.  
2. Print the `.factor` attribute of the object you created. 

Despite the name, the pgmpy `BeliefPropagation` model actually implements the Juncton Tree Algorithm. You can see a more complete description of the algorithm by [reading the documentation](http://pgmpy.org/inference.html#belief-propagation).  

In [11]:
from pgmpy.inference import BeliefPropagation
murder_belief = BeliefPropagation(m_model)
murder_belief.factors

defaultdict(list,
            {'B': [<DiscreteFactor representing phi(B:2) at 0x206bffac518>,
              <DiscreteFactor representing phi(W:2, B:2, C:2) at 0x206bffac390>,
              <DiscreteFactor representing phi(MO:2, B:2, C:2) at 0x206bffac358>,
              <DiscreteFactor representing phi(M:3, B:2, C:2, W:2, MO:2) at 0x206bffac080>],
             'W': [<DiscreteFactor representing phi(W:2, B:2, C:2) at 0x206bffac390>,
              <DiscreteFactor representing phi(M:3, B:2, C:2, W:2, MO:2) at 0x206bffac080>],
             'C': [<DiscreteFactor representing phi(W:2, B:2, C:2) at 0x206bffac390>,
              <DiscreteFactor representing phi(C:2) at 0x206bffac5c0>,
              <DiscreteFactor representing phi(MO:2, B:2, C:2) at 0x206bffac358>,
              <DiscreteFactor representing phi(M:3, B:2, C:2, W:2, MO:2) at 0x206bffac080>],
             'MO': [<DiscreteFactor representing phi(MO:2, B:2, C:2) at 0x206bffac358>,
              <DiscreteFactor representing phi(M:3,

Examine these results, considering the structure of the DAG and its cliques. 

Now, compute a query and print the results for the prior belief (belief with no evidence) for the butler, B, and cook, C, being murderers. 

In [12]:
m_b_prior_q = murder_belief.query(variables=['B', 'C'])
print(m_b_prior_q['B'])
print(m_b_prior_q['C'])

+-----+----------+
| B   |   phi(B) |
+=====+==========+
| B_0 |   0.4000 |
+-----+----------+
| B_1 |   0.6000 |
+-----+----------+
+-----+----------+
| C   |   phi(C) |
+=====+==========+
| C_0 |   0.7000 |
+-----+----------+
| C_1 |   0.3000 |
+-----+----------+


Now, perform the same queries, but include the evidence that the motive what the inheritance (MO = 1) and the weapon was the poison (W = 0). 

In [13]:
m_b_prior_q = murder_belief.query(variables=['B', 'C'], evidence={'MO':1, 'W':0})
print(m_b_prior_q['B'])
print(m_b_prior_q['C'])

+-----+----------+
| B   |   phi(B) |
+=====+==========+
| B_0 |   0.0699 |
+-----+----------+
| B_1 |   0.9301 |
+-----+----------+
+-----+----------+
| C   |   phi(C) |
+=====+==========+
| C_0 |   0.5874 |
+-----+----------+
| C_1 |   0.4126 |
+-----+----------+


Compare the results of the two queries you have just performed with the junction tree algorithm with the equivalent queries you performed with the variable elimination algorithm. Are the results different? Does this outcome surprise you?

ANS: No, the results are the same, which is to be expected since we started with the same model and same evidence, thus our results should be same, irrelevant of which exact inference algorithm we use.

Recall, that there was the possibility that a third party, not the butler or the cook, may have committed the murder. Perform and print a query of the prior marginal of the murderer variable, M.

In [14]:
m_b_prior_q = murder_belief.query(variables=['M'])
print(m_b_prior_q['M'])

+-----+----------+
| M   |   phi(M) |
+=====+==========+
| M_0 |   0.2800 |
+-----+----------+
| M_1 |   0.2100 |
+-----+----------+
| M_2 |   0.5100 |
+-----+----------+


Next, perform and print the query of the murderer variable, M, but with the evidence, MO = 1 and W = 0. 

In [15]:
m_b_evidence_q = murder_belief.query(variables=['M'], evidence={'MO':1, 'W':0})
print(m_b_evidence_q['M'])

+-----+----------+
| M   |   phi(M) |
+=====+==========+
| M_0 |   0.0000 |
+-----+----------+
| M_1 |   0.2413 |
+-----+----------+
| M_2 |   0.7587 |
+-----+----------+


Compare the prior marginal to the marginal including evidence. Is the change in the results consistent with the evidence and why?   

ANS: Since we are given that the third party will not collaborate with either the cook or the butler and since the marginals of both the cook and murderer add up to 100%, that means the third party could not have committed the murder and it's probability drops to 0.

There are many other possible queries you can perform on this model. You may wish to explore some more of these. 